In [7]:
import Neilyst

data = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-01-30T00:00:00Z', timeframe='1h')
data

,open,high,low,close,volume
date,,,,,
2023-01-01 00:00:00+00:00,16537.5,16540.9,16504.0,16527.0,5381.399
2023-01-01 01:00:00+00:00,16527.1,16554.3,16524.1,16550.4,3210.826
2023-01-01 02:00:00+00:00,16550.5,16557.1,16534.8,16542.4,2399.668
2023-01-01 03:00:00+00:00,16542.5,16542.5,16515.0,16529.3,3214.480
2023-01-01 04:00:00+00:00,16529.2,16530.4,16508.8,16517.8,3150.954
...,...,...,...,...,...
2023-12-29 19:00:00+00:00,42090.9,42196.9,41977.5,42152.8,7316.643
2023-12-29 20:00:00+00:00,42152.9,42163.3,41968.5,42073.2,6575.487
2023-12-29 21:00:00+00:00,42073.1,42288.3,41895.7,41974.0,13716.534


In [8]:
# 计算指标

indicators = Neilyst.get_indicators(data, 'rsi', 'sma20', 'sma60', 'sma120')
indicators

,rsi,sma20,sma60,sma120
date,,,,
2023-01-01 00:00:00+00:00,50.476321,16543.325,16634.703333,16713.120000
2023-01-01 01:00:00+00:00,50.476321,16543.325,16634.703333,16713.120000
2023-01-01 02:00:00+00:00,50.476321,16543.325,16634.703333,16713.120000
2023-01-01 03:00:00+00:00,50.476321,16543.325,16634.703333,16713.120000
2023-01-01 04:00:00+00:00,50.476321,16543.325,16634.703333,16713.120000
...,...,...,...,...
2023-12-29 19:00:00+00:00,41.064901,42527.305,42848.208333,42913.773333
2023-12-29 20:00:00+00:00,39.627322,42508.495,42836.868333,42901.700000
2023-12-29 21:00:00+00:00,37.849046,42488.455,42823.516667,42888.843333


In [9]:
data.index[-1]

Timestamp('2023-12-29 23:00:00+0000', tz='UTC')

In [10]:
class MovingAverageStrategy(Neilyst.Strategy):
    def __init__(self, total_balance, trading_fee_ratio, slippage_ratio, data, indicators):
        super().__init__(total_balance, trading_fee_ratio, slippage_ratio, data, indicators)
        self.short_window = 20
        self.long_window = 60

    def run(self, date, price_row, current_pos, current_balance):
        recent_data = self.get_recent_data(date, 2)

        short_mavg = recent_data['sma20'].head(1)
        long_mavg = recent_data['sma60'].head(1)

        prev_short_mavg = recent_data['sma20'].tail(1)
        prev_long_mavg = recent_data['sma60'].tail(1)

        if short_mavg > long_mavg and prev_short_mavg < prev_long_mavg:
            signal = Neilyst.Signal('long', price_row['close'], 1)
        elif short_mavg < long_mavg and prev_short_mavg > prev_long_mavg:
            if current_pos.amount > 0:
                signal = Neilyst.Signal('close', price_row['close'], current_pos.amount)
            else:
                signal = None
        else:
            signal = None
        
        return signal

strategy = MovingAverageStrategy(50000, 0, 0, data, indicators)
result = Neilyst.backtest('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', strategy)
result

Data for 2023-01-01-00:00 to 2023-01-01-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-01-00:00 - 2023-01-01-23:59.csv
Data for 2023-01-02-00:00 to 2023-01-02-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-02-00:00 - 2023-01-02-23:59.csv
Data for 2023-01-03-00:00 to 2023-01-03-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-03-00:00 - 2023-01-03-23:59.csv
Data for 2023-01-04-00:00 to 2023-01-04-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-04-00:00 - 2023-01-04-23:59.csv
Data for 2023-01-05-00:00 to 2023-01-05-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-05-00:00 - 2023-01-05-23:59.csv
Data for 2023-01-06-00:00 to 2023-01-06-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-06-00:00 - 2023-01-06-23:59.csv
Data for 2023-01-07-00:00 to 2023-01-07-23:59 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/1m/2023-01-07-00:00 - 2023-01-07-23

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().